
# 👾 PixelCNN from scratch（PyTorch 版本）

在这个 Notebook 中，我们将**从零开始**，使用 **PyTorch** 在 **Fashion-MNIST** 数据集上训练一个 **PixelCNN** 模型。


## 0. 参数（Parameters）

In [ ]:

IMAGE_SIZE = 16          # 输入图像大小
PIXEL_LEVELS = 4         # 像素离散级别
N_FILTERS = 128          # 卷积通道数
RESIDUAL_BLOCKS = 5      # Residual Block 数量
BATCH_SIZE = 128
EPOCHS = 150
LR = 5e-4


## 1. 数据准备（Prepare the data）

In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms

import numpy as np
import matplotlib.pyplot as plt


In [ ]:

# 加载 Fashion-MNIST（仅使用训练集图像）
train_dataset = datasets.FashionMNIST(
    root="./data",
    train=True,
    download=True,
    transform=transforms.ToTensor(),
)

x_train = train_dataset.data.numpy()  # (N, 28, 28)


In [ ]:

# 数据预处理：
# 1. resize 到 IMAGE_SIZE x IMAGE_SIZE
# 2. 离散化像素到 [0, PIXEL_LEVELS-1]
# 3. 归一化到 [0, 1]

def preprocess(imgs_int):
    imgs_int = np.expand_dims(imgs_int, axis=1)  # (N,1,H,W)
    imgs_int = torch.tensor(imgs_int, dtype=torch.float32)
    imgs_int = F.interpolate(
        imgs_int, size=(IMAGE_SIZE, IMAGE_SIZE), mode="nearest"
    )
    imgs_int = (imgs_int / (256 / PIXEL_LEVELS)).long()
    imgs = imgs_int.float() / PIXEL_LEVELS
    return imgs, imgs_int

input_data, output_data = preprocess(x_train)


In [ ]:

# 构建 DataLoader
dataset = TensorDataset(input_data, output_data.squeeze(1))
loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)


In [ ]:

# 可视化部分训练样本
def display(images, n=10):
    images = images[:n]
    fig, axes = plt.subplots(1, n, figsize=(n * 1.5, 1.5))
    for i, ax in enumerate(axes):
        ax.imshow(images[i, 0], cmap="gray")
        ax.axis("off")
    plt.show()

display(input_data)


## 2. 构建 PixelCNN 模型（Build the PixelCNN）

In [ ]:

class MaskedConv2d(nn.Conv2d):
    def __init__(self, mask_type, *args, **kwargs):
        super().__init__(*args, **kwargs)
        assert mask_type in ["A", "B"]
        self.mask_type = mask_type
        self.register_buffer("mask", torch.ones_like(self.weight))

        _, _, h, w = self.weight.shape
        self.mask[:, :, h // 2, w // 2 + (mask_type == "B") :] = 0
        self.mask[:, :, h // 2 + 1 :, :] = 0

    def forward(self, x):
        self.weight.data *= self.mask
        return super().forward(x)


In [ ]:

class ResidualBlock(nn.Module):
    def __init__(self, filters):
        super().__init__()
        self.conv1 = nn.Conv2d(filters, filters // 2, kernel_size=1)
        self.conv2 = MaskedConv2d(
            "B",
            filters // 2,
            filters // 2,
            kernel_size=3,
            padding=1,
        )
        self.conv3 = nn.Conv2d(filters // 2, filters, kernel_size=1)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.relu(self.conv2(out))
        out = F.relu(self.conv3(out))
        return x + out


In [ ]:

class PixelCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.input_conv = MaskedConv2d(
            "A", 1, N_FILTERS, kernel_size=7, padding=3
        )

        self.res_blocks = nn.ModuleList(
            [ResidualBlock(N_FILTERS) for _ in range(RESIDUAL_BLOCKS)]
        )

        self.conv1 = MaskedConv2d("B", N_FILTERS, N_FILTERS, kernel_size=1)
        self.conv2 = MaskedConv2d("B", N_FILTERS, N_FILTERS, kernel_size=1)

        self.out_conv = nn.Conv2d(N_FILTERS, PIXEL_LEVELS, kernel_size=1)

    def forward(self, x):
        x = F.relu(self.input_conv(x))
        for block in self.res_blocks:
            x = block(x)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        return self.out_conv(x)


## 3. 训练 PixelCNN（Train the PixelCNN）

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = PixelCNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss()


In [ ]:

def generate_images(model, num_img=10, temperature=1.0):
    model.eval()
    images = torch.zeros(
        num_img, 1, IMAGE_SIZE, IMAGE_SIZE, device=device
    )

    with torch.no_grad():
        for i in range(IMAGE_SIZE):
            for j in range(IMAGE_SIZE):
                logits = model(images)[:, :, i, j]
                probs = F.softmax(logits / temperature, dim=1)
                samples = torch.multinomial(probs, 1).squeeze()
                images[:, 0, i, j] = samples.float() / PIXEL_LEVELS

    return images.cpu()


In [ ]:

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0.0

    for x, y in loader:
        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch:03d} | Loss: {total_loss / len(loader):.4f}")

    # 每个 epoch 生成样本
    samples = generate_images(model, num_img=10)
    display(samples)


## 4. 图像生成（Generate images）

In [ ]:

samples = generate_images(model, num_img=10, temperature=1.0)
display(samples)
